## О задании
В этом задании вы будете прогнозировать отток клиентов банка с помощью ансамблевых методов. Работать будем с датасетом [Churn for Bank Customers](https://www.kaggle.com/datasets/mathchi/churn-for-bank-customers)
(подробнее с описанием признаков можете ознакомиться по ссылке). Целевая переменная - `Exited` (1 - клиент покинул банк, 0 - клиент не покинул банк)

### Формат сдачи
Данное задание сдаётся через [эту гугл форму](https://forms.gle/jrdU8j2zH6KQ8BATA). Не забудьте открыть доступ по ссылке к файлу (справа сверху "Поделиться" или "Share")

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Загрузим данные. 

Данные можно найти по [ссылке](https://drive.google.com/file/d/1rz00kErCbYlRh5Y0zdYENN8nlixAyLlh/view?usp=share_link)

In [2]:
df = pd.read_csv('./resources/churn.csv')

In [3]:
df.shape

(10000, 14)

In [4]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


Выведите сводные характеристики данных, воспользовавшись методом `.describe()`.

In [5]:
df.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


**Вопрос.** Какая задача машинного обучения решается?

<u>**Ответ**</u>: классификация 

Посмотрите на соотношение классов в целевой переменной `Exited` (выведите, сколько элементов каждого класса или в каком они соотношении)

In [6]:
display(df['Exited'].value_counts())
display(df['Exited'].value_counts(normalize=True))

0    7963
1    2037
Name: Exited, dtype: int64

0    0.7963
1    0.2037
Name: Exited, dtype: float64

**Вопрос.** Присутствует ли дисбаланс классов?

<u>**Ответ**</u>: Да, присутствует сильный дисбаланс классов, 1-ый класс имеют ~80% выборки

**Вопрос.** Если есть дисбаланс классов, то как с ним справляются? Приведите способы, как обходиться с ним

<u>**Ответ**</u>: 

способы:

1. Использовать подходящие метрики качества (Precision/Recall, F-score).   
2. Дублирование наблюдений на основе имеющихся для классов с меньшим числом наблюдений или 
уменьшение наблюдений до числа элементов в меньшем классе.
3. Использование ансамблей/деревьев, других моделей, которые хорошо работают на несбалансированной выборке

**Вопрос.** Какая метрика важнее в данной задаче: precision или recall? Обоснуйте свой выбор

<u>**Ответ**</u>: в данной задаче, скорее всего, recall (в идеале, конечно, f1)

Все зависит от задач банка, так, например, если нужно просто понять количество клиентов, которые хотят уйти,
то использование recall более подходящее, так как если оптимизировать по precision, то мы можем сильно недооценить ситуацию. 
Однако, представим, что банк хочет попытаться удержать клиентов,
которые планируют уйти, для этого он может предлагать клиентам различные акции. Если мы будем иметь хороший recall, но низкий
precision, то банк может понести большие издержки, предложив слишком много лишних акций. С другой стороны, если маленький recall,
но низкий precision, банк предложит акции лишь малой группе планирующих уходить людей. Поэтому следует исходить из того, какие издержки выше.


## Отбор признаков для модели

Переведите значения признака `Gender` в числа. В результате должен получиться столбец из 0 и 1

In [7]:
df['Gender'] = df['Gender'].astype('category').cat.codes 
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


Отберите признаки, на которых вы будете обучать модели. Учтите, что будут обучаться ансамбли из решающих деревьев. Запишите в переменную `X` независимые признаки, а в переменную `y` - целевую переменную. Если необходимо, сделайте кодирование категориальные признаков в числовые.

In [8]:
### ЗДЕСЬ ВАШ КОД ¯\_(ツ)_/¯ 
feats = ['Age', 'CreditScore', 'Gender', 'Balance', 'Geography', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'Tenure', 'EstimatedSalary'] # сюда добавьте признаки, на основе которых будет строится прогноз
df['Geography'] = df['Geography'].astype('category').cat.codes 

X = df[feats]
y = df['Exited']

## Разбиение данных

Разбейте данные на тренировочные и тестовые

*Не забудьте при разбиении сохранить соотношение классов в тренировочной и тестовой выброках

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=1337) 

## Моделирование

Мы будем обучать ансамблевые методы. Среди них: случайный лес и градиентный бустинг

**Вопрос.** Чем отличается алгоритм случайного леса и градиентный бустинг?

<u>**Ответ**</u>: оба алгоритма являются ансамблями, однако случайный лес усредняет результат результат работы нескольких решающих деревьев (они независимы), 

а в градиентном бустинге модели (не обязательно деревья) последовательно обучаются на ошибках прошлой модели.

### Обучение случайного леса

In [11]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

Обучите модель случайного леса на `X_train`, `y_train`

In [12]:
clf = RandomForestClassifier(n_jobs=-1)
clf.fit(X_train, y_train)

RandomForestClassifier(n_jobs=-1)

#### Оценка модели случайного леса

Сделайте предсказание обученной модели на `X_test` и оцените precision, recall и f1_score, имея реальные метки `y_test`

In [13]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [14]:
### ЗДЕСЬ ВАШ КОД ¯\_(ツ)_/¯ 

y_pred = clf.predict(X_test) # предсказания на X_test
precision = precision_score(y_test, y_pred) # значение метрики precision
recall = recall_score(y_test, y_pred) # значение метрики recall
f1 = f1_score(y_test, y_pred) # значение метрики f1

for metric, metric_name in zip([precision, recall, f1], ['precision', 'recall', 'f1']):
    print(f'{metric_name}: {metric}')

precision: 0.8134556574923547
recall: 0.4353518821603928
f1: 0.5671641791044776


**(*) Дополнительно.** Попробуйте варьировать различные гиперпараметры случайного леса (как минимум, число деревьев), а также попробуйте заменить `RandomForestClassifier` на `ExtraTreesClassifier`. Повлияло ли это на метрики? Опишите результаты

<u>**Ответ**</u>: _ _ _ _ _ _ _ _ _ _ _ _

### Обучение градиентного бустинга

Обучите модель градиентного бустинга на тех же данных

In [15]:
from sklearn.ensemble import GradientBoostingClassifier

In [16]:
### ЗДЕСЬ ВАШ КОД ¯\_(ツ)_/¯ 

clf = GradientBoostingClassifier()
clf.fit(X_train, y_train)

GradientBoostingClassifier()

Замерьте те же метрики, что и при случайном лесе

In [17]:
y_pred = clf.predict(X_test) # предсказания на X_test
precision = precision_score(y_test, y_pred) # значение метрики precision
recall = recall_score(y_test, y_pred) # значение метрики recall
f1 = f1_score(y_test, y_pred) # значение метрики f1

for metric, metric_name in zip([precision, recall, f1], ['precision', 'recall', 'f1']):
    print(f'{metric_name}: {metric}')

precision: 0.8363636363636363
recall: 0.45171849427168576
f1: 0.5866099893730073


**(*) Дополнительно.** Попробуйте варьировать различные гиперпараметры градиентного бустинга (как минимум, число деревьев и скорость обучения) Есть ли прирост в качестве? Опишите результаты

<u>**Ответ**</u>: _ _ _ _ _ _ _ _ _ _ _ _

**Вопрос.** Сравните полученные результаты. Какая модель оказалась лучше по метрике f1?

<u>**Ответ**</u>: модель градиентного бустинга показала наилучший результат

### Перебор гиперпараметров

Выберите модель, у которой вы будете подбирать гиперпараметры

### Я выбрал(-а) <u>**градиентный бустинг**</u>

Перейдите на страницу выбранного алгоритма для нахождения гиперпараметров, которые можно перебирать: [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)/[ExtraTreesClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html), [GradientBoostingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html)

Для подбора гиперпараметров будем использовать [RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html), который вместо всех комбинаций гиперпараметров перебирает заданное число случайных комбинаций 

In [18]:
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold

Инициализируйте `RandomizedSearchCV`. Основные моменты:
* Укажите число случайных комбинаций гиперпараметров `n_iter`
* Необходимо добавить в `params` гиперпараметры, которые будут перебираться, а также множество значений их гиперпараметров
* В `RandomizedSearchCV` передайте:
    * экземпляр выбранной модели для классификации
    * `n_iter`
    * стратегию разбиение данных на кросс-валидацию `cv`
    * скоринговую функцию `scoring` (строка 'precision' или 'recall', в зависимости от важности для нас той или иной метрики)

In [19]:
### ЗДЕСЬ ВАШ КОД ¯\_(ツ)_/¯ 

clf = GradientBoostingClassifier() # инициализируйте выбранную модель (случайного леса или градиентного бустинга)

n_iter = 250 # число случайных комбинаций гиперпараметров

params = {
    'n_estimators': [100, 150, 200, 250],
    'criterion': ['friedman_mse', 'squared_error'],
    'min_samples_leaf': np.arange(1, 10, 1),
    'max_depth': [3, 5, 7],
    'min_samples_split': np.arange(2, 6),
    'max_features': [None, 'sqrt', 'log2']
}

skf = StratifiedKFold(5, shuffle=True, random_state=0)

grid = RandomizedSearchCV(
    clf,
    param_distributions=params,
    n_iter=n_iter,
    cv=skf,
    scoring='recall', # 'precision' или 'recall'
    verbose=-1,
    n_jobs=-1
)

Запустите обучение. Не забывайте, что обучаемся на тренировочной выборке

In [20]:
grid.fit(X_train, y_train)

RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=0, shuffle=True),
                   estimator=GradientBoostingClassifier(), n_iter=250,
                   n_jobs=-1,
                   param_distributions={'criterion': ['friedman_mse',
                                                      'squared_error'],
                                        'max_depth': [3, 5, 7],
                                        'max_features': [None, 'sqrt', 'log2'],
                                        'min_samples_leaf': array([1, 2, 3, 4, 5, 6, 7, 8, 9]),
                                        'min_samples_split': array([2, 3, 4, 5]),
                                        'n_estimators': [100, 150, 200, 250]},
                   scoring='recall', verbose=-1)

Выведите лучшие найденные гиперпараметры, обратившись к `grid`. Воспользуйтесь атрибутами `best_score_` и `best_params_`

In [21]:
### ЗДЕСЬ ВАШ КОД ¯\_(ツ)_/¯ 
display(grid.best_score_)
display(grid.best_params_)

0.4754484112378849

{'n_estimators': 250,
 'min_samples_split': 2,
 'min_samples_leaf': 6,
 'max_features': 'log2',
 'max_depth': 5,
 'criterion': 'friedman_mse'}

Сделайте предсказание на тестовой выборке и замерьте метрики

In [22]:
y_pred = grid.predict(X_test) 
precision = precision_score(y_test, y_pred) # значение метрики precision
recall = recall_score(y_test, y_pred) # значение метрики recall
f1 = f1_score(y_test, y_pred) # значение метрики f1

for metric, metric_name in zip([precision, recall, f1], ['precision', 'recall', 'f1']):
    print(f'{metric_name}: {metric}')

precision: 0.7684210526315789
recall: 0.4779050736497545
f1: 0.5893037336024217


**Вопрос.** Улучшилось ли качество модели после подбора гиперпараметров?

<u>**Ответ**</u>: Получилось немного увеличить recall, precision при этом упал. 
F1 score вырос

Если нет значимых изменений в качестве, то имеет смысл:
* Увеличить число случайных комбинаций `n_iter` (придётся подождать больше времени)
* Добавить другие гиперпараметры
* Расширить множество перебираемых значений гиперпараметров
* Запустить заново `grid.fit` для перебора других случайных комбинаций

### (*) Дополнительно

**(*) Дополнительно.** Возьмите алгоритм `XGBClassifier` из библиотеки `xgboost` и обучите его на данной задаче, подберите гиперпараметры и замерьте метрики. Сравните алгоритм по скорости и полученному качеству

<u>**Ответ**</u>: _ _ _ _ _ _ _ _ _ _ _ _

**(*) Дополнительно.** Возьмите алгоритм `CatBoostClassifier` из библиотеки `catboost` и обучите его на данной задаче, можете добавить параметр для отображения графиков во время обучения, подберите гиперпараметры и замерьте метрики. Сравните алгоритм по скорости и полученному качеству

<u>**Ответ**</u>: _ _ _ _ _ _ _ _ _ _ _ _

**(*) Дополнительно.** Возьмите алгоритм стекинга [`StackingClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingClassifier.html), обучите его и замерьте качество. В качестве базовых алгоритмов можете брать KNeighborsClassifier, RandomForestClassifier, LogisticRegression, BernoulliNB,.. В качестве мета алгоритма возьмите GradientBoostingClassifier или XGBClassifier. 

Получилось ли обучить его? Какие проблемы возникли? Какое качество в итоге получено?

<u>**Ответ**</u>: _ _ _ _ _ _ _ _ _ _ _ _

## Выводы

**Вопрос.** Напишите выводы о проделанной работе. Должны содержаться ответы на следующие вопросы:
* Какая модель показала лучшее качество?
* Получилось ли добиться лучшего качества после подбора гиперпараметров?

<u>**Ответ**</u>: была проведена работа по использованию ансамблевых алгоритмов для решение задачи классификации
при несбалансированных классах.
При стандартных параметрах из sklearn градиментный бустинг оказался лучше, чем случайный лес. Использование
перебора по сетке позволило улучшить качество целевой метрики (recall), а также f1 score, но также получили 
меньший precision. 